In [80]:
import pandas as pd
import altair as alt

In [81]:
dados_painelfederal_campina = pd.read_csv("data\\dados_painel-federal_obitos_cg.csv", encoding='UTF-8', error_bad_lines=False)
dados_municipais_campina = pd.read_csv("data\\dados_municipais_cg.csv", encoding='utf-8', index_col='data')

# Processando os dados sobre Casos Confirmados por Dia

 ## Dados Federais

In [82]:
# Criando um data com a coluna 'data' e a coluna 'casosNovos'
df_casosPorDia = dados_painelfederal_campina[['data', 'casosNovos']]

In [83]:
# Criando um DataFrame com os dados para o altair
df_casosPorDia = pd.DataFrame({
    'data': df_casosPorDia['data'].tolist(),
    'casos_confirmados': df_casosPorDia['casosNovos'].tolist()
})
# Criando uma nova coluna no DataFrame referente a média movel.
df_casosPorDia['media_movel'] = df_casosPorDia.casos_confirmados.rolling(7).mean()

# Setando a coluna 'data' como índice do DataFrame 
df_casosPorDia.set_index('data', inplace=True)

## Dados Municipal

In [84]:
# Criando um data com a coluna 'confirmadosPorDia'
# OBS: 'data' já é o index
dm_casosPorDia = dados_municipais_campina['confirmadosPorDia']

In [85]:
# Criando um DataFrame com os dados para o altair
dm_casosPorDia = pd.DataFrame({
    'data': dm_casosPorDia.index.tolist(),
    'casos_confirmados': dm_casosPorDia.values.tolist()
})
# Criando uma nova coluna no DataFrame referente a média movel.
dm_casosPorDia['media_movel'] = dm_casosPorDia.casos_confirmados.rolling(7).mean()

# Setando a coluna 'data' como índice do DataFrame 
dm_casosPorDia.set_index('data', inplace=True)

# Excluindo as datas não contidas nos DataFrames (Federal e Municipal)

In [86]:
# Função que exclui os elementos (pelo index) do DataFrame que não estão em outro DataFrame
def exclui_nao_contidos(df_conjunto, df_subconjunto):
    for i in df_conjunto.index:
        if i not in df_subconjunto.index:
            df_conjunto = df_conjunto.drop(i)
    
    return df_conjunto

In [87]:
# Excluindo as datas que estão nos dados federais mas não estão nos dados municipais
df_casosPorDia = exclui_nao_contidos(df_casosPorDia, dm_casosPorDia)

# Excluindo as datas que estão nos dados municipais mas não estão nos dados federais
dm_casosPorDia = exclui_nao_contidos(dm_casosPorDia, df_casosPorDia)

# Transformando os DataFrame do formato wide para o formato long

In [88]:
# Função que transforma os DataFrames do formato wide para o formato long
def to_longFormat(dt_federal, dt_municipal, nome_coluna):
    long_df = pd.DataFrame({
    'data': dt_federal.index.tolist(),
    'Federal': dt_federal[nome_coluna].tolist(),
    'Municipal': dt_municipal[nome_coluna].tolist()
    }).melt(
        id_vars=['data'], 
        value_vars=['Federal', 'Municipal'],
        var_name='fonte_dados', 
        value_name='casos_por_dia'
    )
    return long_df

In [89]:
# Transformando os DataFrames sobre casos confirmados
casosPorDia = to_longFormat(df_casosPorDia, dm_casosPorDia, 'casos_confirmados')

# Transformando os DataFrames sobre a média movel dos casos confirmados
casosPorDiaMedMovel = to_longFormat(df_casosPorDia, dm_casosPorDia, 'media_movel')

# Comparando os dados Federais e Municipais

In [112]:
# Função que cria os graficos de linhas (mark_line) no altair
def criar_mark_line(dataframe, titulo, largura, pontos):
    mark_line = alt.Chart(dataframe).mark_line(point=pontos).encode(
                    alt.X('data:T', title='Data'),
                    alt.Y('casos_por_dia:Q', title='Nº de Casos'),
                    alt.Color('fonte_dados:N', title='Base dos Dados')
                ).properties(
                    width=largura,
                    title=titulo
                )
    return mark_line

### Casos Confirmados por Dia

In [113]:
criar_mark_line(casosPorDia, 'Quantidade de Casos Confirmados por Dia', 1000, False)

alt.Chart(...)

### Média Movel (7 dias) dos Casos Confirmados

In [114]:
criar_mark_line(casosPorDiaMedMovel, 'Média Movel de Casos Confirmados por Dia', 1000, True)

alt.Chart(...)

### Total de Casos Confirmados

In [115]:
df_casosTotais = df_casosPorDia['casos_confirmados'].sum()
dm_casosTotais = dm_casosPorDia['casos_confirmados'].sum()

In [116]:
totalCasos = pd.DataFrame({
    'base': ['Federal', 'Municipal'],
    'total': [df_casosTotais, dm_casosTotais]
})

In [117]:
alt.Chart(totalCasos).mark_bar(size=30).encode(
    alt.X('base:N', title='Base dos Dados'),
    alt.Y('total:Q', title='Nº Total de Casos'),
    alt.Color('base:N', title='Base dos Dados')
).properties(
    width=250,
    title='Total de Casos Confirmados'
)

alt.Chart(...)